In [1]:
import tkinter as tk
import socket
import os
from datetime import datetime
HOST = "192.168.30.228"   # 改成你的 Ingest IP
PORT = 32108             # 改成你的 Port


In [16]:
date_folder = datetime.today().strftime("%m.%d.%Y")
date_prefix = datetime.today().strftime("%m%d")  

In [17]:
SAVE_BASE = r"E:\\"
folder_path = os.path.join(SAVE_BASE, date_folder)

In [18]:
folder_path = os.path.join(SAVE_BASE, date_folder)
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    log(f"📁 建立新資料夾：{folder_path}")
else:
    log(f"📁 使用現有資料夾：{folder_path}")

📁 建立新資料夾：E:\\06.26.2025


In [20]:
final_filename = f"{date_prefix}_中文"  # 0626_使用者輸入
full_path = os.path.join(date_folder, final_filename)  # 不要含 E:\ 以免多餘

In [6]:
import socket
def connect_socket():
    HOST = "192.168.30.228"
    PORT = 32108
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.settimeout(3)
        log(f"🚀 嘗試連線 {HOST}:{PORT}...")
        s.connect((HOST, PORT))
        log("✅ 連線成功")
        return s  # 傳回已連線的 socket 物件
    except Exception as e:
        log("❌ 無法連線:", e)
        return None
def send_command(sock, cmd):
    try:
        sock.sendall((cmd + "\r\n").encode("cp950", errors="replace"))
        response = sock.recv(2048).decode("cp950", errors="replace")
        log(f"⬅️ Response:\n{response}")
        return response.strip()
    except Exception as e:
        log("❌ 傳送/接收失敗:", e)
        return ""
def list_encoders():
    response = send_command("List")
    if not response:
        log("⚠️ 沒收到任何回應")
        return []

    lines = response.splitlines()
    encoder_names = []
    for line in lines:
        line = line.strip()
        if line and "Mode:" in line:
            # 例如：Bak4-1   Mode: Quick
            encoder_name = line.split("Mode:")[0].strip()
            encoder_names.append(encoder_name)

    log(f"🎯 擷取到 Encoder 名稱：{encoder_names}")
    return encoder_names


In [26]:
# encoder_utils.py
import socket

HOST = "192.168.30.228"
PORT = 32108

def connect_socket():
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.settimeout(3)
        s.connect((HOST, PORT))
        log("✅ 已建立 socket 連線")
        return s
    except Exception as e:
        log("❌ 連線失敗:", e)
        return None

def send_command(sock, cmd):
    try:
        encoded = (cmd + "\r\n").encode("cp950")
        sock.sendall(encoded)
        sock.settimeout(2)
        data = b""
        while True:
            try:
                chunk = sock.recv(1024)
                if not chunk:
                    break
                data += chunk
            except socket.timeout:
                break
        response = data.decode("cp950", errors="replace")
        log("⬅️ Response:\n", response)
        return response.strip()
    except Exception as e:
        log("❌ 指令傳送失敗:", e)
        return ""

def list_encoders():
    sock = connect_socket()
    if not sock:
        return []
    response = send_command(sock, "List")
    sock.close()

    encoders = []
    for line in response.splitlines():
        if "Mode:" in line:
            enc_name = line.split("Mode:")[0].strip()
            encoders.append(enc_name)
    if not encoders:
        log("⚠️ 沒有從 socket 抓到 encoder")
    return encoders


In [28]:

encoder_names = list_encoders()
log("✅ Encoder 列表：", encoder_names)


✅ 已建立 socket 連線
⬅️ Response:
  
OK: 
Bak4-1      Mode: Quick
State: Runned
Start Time: 2025/6/27 下午 05:48:33
   1920x1080 29.97 fps,      8 Ch. 48000 Hz
   No stream
Video Signal: Exists
Bak4-2      Mode: Quick
State: Runned
Start Time: 2025/6/27 下午 05:48:33
   1920x1080 29.97 fps,      8 Ch. 48000 Hz
   No stream
Video Signal: Exists


✅ Encoder 列表： ['Bak4-1', 'Bak4-2']


In [25]:
encoder_names = list_encoders()

✅ 已建立 socket 連線
⬅️ Response:

⚠️ 沒有從 socket 抓到 encoder
